In [1]:
import os
import sys
current_folder = os.path.dirname(os.path.abspath('__file__'))
parent_folder = os.path.dirname(current_folder)
grandparent_folder = os.path.dirname(parent_folder)
grandgrandparent_folder = os.path.dirname(grandparent_folder)
grandgrandgrandparent_folder = os.path.dirname(grandgrandparent_folder)
sys.path.append(grandgrandgrandparent_folder)

In [2]:
import optuna
from optuna.trial import TrialState
from optuna.samplers import TPESampler

import torch
import torch.optim as optim
import torch.nn as nn

import numpy as np
import sklearn
import copy

from methods.PINN import PINN
from methods.methodsDataset.PINNDataset import PINNDataset
from methods.DataDrivenMethods import DDMethod
from solvers.Solver import Solver

In [3]:
params_solver = {'equation': 'Poisson', 'domain': [0., 1.], 'D': 1., 'nx': 101}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
params_PINN = {'layer_dims': None, 'activations': None, 'device': device, 'seed': 123}

In [4]:
Dmin, Dmax = 0.1, 10
D_list = np.linspace(Dmin, Dmax, 1000)

U_sols = []
solver = Solver(params={'solver': params_solver})
for d in D_list:
    solver.change_D(new_D=d)
    U_sols.append(solver.solve())

U_sols = np.stack(U_sols)
D = torch.Tensor(D_list).view(-1, 1).to(device)
U = torch.Tensor(U_sols).to(device)

In [5]:
d_train, d_val, u_train, u_val = sklearn.model_selection.train_test_split(D_list, U_sols, test_size=0.2, 
                                                                          random_state=params_PINN['seed'])

D_train_repeated = torch.Tensor([[d] * len(solver.x) for d in d_train]).view(-1, 1)
D_val_repeated = torch.Tensor([[d] * len(solver.x) for d in d_val]).view(-1, 1)
x = torch.Tensor(solver.x).view(-1, 1)
X_train = x.repeat(d_train.shape[0], 1)
X_val = x.repeat(d_val.shape[0], 1)

DX_train = torch.cat((D_train_repeated, X_train), dim=1)
DX_val = torch.cat((D_val_repeated, X_val), dim=1)

In [6]:
def get_dataset():
    return DX_train, DX_val

In [7]:
def loss_fn(x, y = 0):
    return torch.square(y - x).mean()

In [8]:
def define_model(trial, input_size, output_size):
    n_layers = trial.suggest_int("n_layers", 1, 10)
    layers = [input_size]
    activations = []
    for i in range(n_layers):
        out_features = trial.suggest_int("units_l{}".format(i), 24, 48)
        activation = trial.suggest_categorical("activation_l{}".format(i), ["tanh", "softplus", "relu"])
        layers += [out_features]
        activations.append(activation)
    layers += [output_size]
    params_PINN_trial = copy.deepcopy(params_PINN)
    params_PINN_trial['layer_dims'] = layers
    params_PINN_trial['activations'] = activations
    return PINN(params={'solver':None, 'method':params_PINN_trial})

In [9]:
def objective(trial):
    # Generate the model.
    model = define_model(trial, 2, 1)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    # optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    DX_train, DX_val = get_dataset()

    trainDataset = PINNDataset(x=DX_train)
    valDataset = PINNDataset(x=DX_val)

    batch_size = trial.suggest_int("batch_size", 20000, 40000)
    torch.manual_seed(params_PINN['seed'])
    # trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=batch_size, shuffle=True)
    # valLoader = torch.utils.data.DataLoader(valDataset, batch_size=batch_size, shuffle=False)

    hyperparameters = {'lr': lr, 'epochs': 10, 'optimizer': optimizer_name, 'batch_size': batch_size}
    # Training of the model.
    model.fit(hyperparameters = hyperparameters, DX_train = DX_train, DX_val = DX_val)
    
    val_loss = model.losses['val']
    val_tot = [sum(x) for x in zip(val_loss['ic_bc'], val_loss['residual'])]
    trial.report(min(val_tot), 10)

    # Handle pruning based on the intermediate value.
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

    return min(val_tot)

In [10]:
sampler = TPESampler(seed=params_PINN['seed'])
study = optuna.create_study(direction="minimize", sampler=sampler)

[I 2023-08-03 09:16:00,597] A new study created in memory with name: no-name-cae4691f-cf6c-47e7-871b-4cbd1d4e390d


In [11]:
study.optimize(objective, n_trials=10, timeout=600)

[I 2023-08-03 09:16:50,448] Trial 0 finished with value: 0.7204361706972122 and parameters: {'n_layers': 7, 'units_l0': 31, 'activation_l0': 'relu', 'units_l1': 34, 'activation_l1': 'tanh', 'units_l2': 33, 'activation_l2': 'softplus', 'units_l3': 25, 'activation_l3': 'softplus', 'units_l4': 28, 'activation_l4': 'relu', 'units_l5': 45, 'activation_l5': 'tanh', 'units_l6': 32, 'activation_l6': 'tanh', 'optimizer': 'Adam', 'lr': 0.0005289943741212551, 'batch_size': 29874}. Best is trial 0 with value: 0.7204361706972122.
[I 2023-08-03 09:17:36,173] Trial 1 finished with value: 48.7509845495224 and parameters: {'n_layers': 5, 'units_l0': 31, 'activation_l0': 'relu', 'units_l1': 36, 'activation_l1': 'tanh', 'units_l2': 34, 'activation_l2': 'tanh', 'units_l3': 48, 'activation_l3': 'softplus', 'units_l4': 44, 'activation_l4': 'tanh', 'optimizer': 'SGD', 'lr': 0.03175611508579144, 'batch_size': 30208}. Best is trial 0 with value: 0.7204361706972122.
[I 2023-08-03 09:18:32,298] Trial 2 finished 

In [12]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  10
  Number of pruned trials:  4
  Number of complete trials:  6
Best trial:
  Value:  0.7204361706972122
  Params: 
    n_layers: 7
    units_l0: 31
    activation_l0: relu
    units_l1: 34
    activation_l1: tanh
    units_l2: 33
    activation_l2: softplus
    units_l3: 25
    activation_l3: softplus
    units_l4: 28
    activation_l4: relu
    units_l5: 45
    activation_l5: tanh
    units_l6: 32
    activation_l6: tanh
    optimizer: Adam
    lr: 0.0005289943741212551
    batch_size: 29874


In [13]:
activations = []
layer_dims = []

for key, value in trial.params.items():
    if key.split('_')[0] == 'activation':
        activations.append(value)
    elif key.split('_')[0] == 'units':
        layer_dims.append(value)
    elif key == 'optimizer':
        optimizer = value
    elif key == 'lr':
        lr = value

Use this to fill config_step_1.py

In [14]:
print(activations, '\n', layer_dims, '\n', optimizer, '\n', lr)

['relu', 'tanh', 'softplus', 'softplus', 'relu', 'tanh', 'tanh'] 
 [31, 34, 33, 25, 28, 45, 32] 
 Adam 
 0.0005289943741212551
